In [40]:
import wrangle
import acquire
import unicodedata
import re

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
import env


import unicodedata
import re
import json

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

import pandas as pd
import acquire
from time import strftime

import warnings
warnings.filterwarnings('ignore')

# import acquire as a
# from env import github_token, github_username
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
from sklearn.model_selection import train_test_split
import os

## Bring in the data

In [4]:
python_df = acquire.get_github_python_data()

In [5]:
python_df.head()

,repo_name,url,language,readme_content
0,jackfrued/Python-100-Days,https://github.com/jackfrued/Python-100-Days/b...,Python,Python - 100天从新手到大师作者：骆昊说明：从项目上线到获得8w+星标以来，一直收...
1,donnemartin/system-design-primer,https://github.com/donnemartin/system-design-p...,Python,English ∙ 日本語 ∙ 简体中文 ∙ 繁體中文 | العَرَبِيَّة‎ ∙ ...
2,Significant-Gravitas/Auto-GPT,https://github.com/Significant-Gravitas/Auto-G...,Python,Auto-GPT: An Autonomous GPT-4 Experiment💡 Get ...
3,public-apis/public-apis,https://github.com/public-apis/public-apis/blo...,Python,Public APIs A collective list of free A...
4,AUTOMATIC1111/stable-diffusion-webui,https://github.com/AUTOMATIC1111/stable-diffus...,Python,Stable Diffusion web UIA browser interface bas...


In [4]:
# type(python_df.readme_content[3])

str

In [5]:
# javascript_df = acquire.get_github_java_script_data()

In [6]:
# javascript_df.head()

## Test the prepare function

In [7]:
# train, validate, test = wrangle.wrangle_readmes()

In [8]:
def clean(string):
    """
    This function puts a string in lowercase, normalizes any unicode characters, removes anything that         
    isn't an alphanumeric symbol or single quote.
    """
    # Normalize unicode characters
    string = unicodedata.normalize('NFKD', string)\
    .encode('ascii', 'ignore')\
    .decode('utf-8', 'ignore')
    
    # Remove unwanted characters and put string in lowercase
    string = re.sub(r"[^\w0-9'\s]", '', string).lower()
            
    return string

In [33]:
def lemmatize(string):
    """
    This function takes in a string, lemmatizes each word, and returns a lemmatized version of the orignal string
    """
    # Build the lemmatizer
    lemmatizer = nltk.stem.WordNetLemmatizer()
    
    results = []
    for word in string.split():
        results.append(lemmatizer.lemmatize(word))
    
    # Convert back into a string
    string = ' '.join(results)

    return string

In [36]:
def remove_stopwords(string, extra_words=None, exclude_words=None):
    """
    Takes in a string, with optional arguments for words to add to stock stopwords and words to ignore in the 
    stock list removes the stopwords, and returns a stopword free version of the original string
    """
    # Get the list of stopwords from nltk
    stopword_list = stopwords.words('english')
    
    # Create a set of stopwords to exclude
    excluded_stopwords = set(exclude_words) if exclude_words else set()
    
    # Include any extra words in the stopwords to exclude
    stopwords_to_exclude = set(stopword_list) - excluded_stopwords
    
    # Add extra words to the stopwords set
    stopwords_to_exclude |= set(extra_words) if extra_words else set()
    
    # Tokenize the input string
    words = string.split()
    
    # Filter out stopwords from the tokenized words
    filtered_words = [word for word in words if word not in stopwords_to_exclude]
    
    # Convert back to string
    string = ' '.join(filtered_words)
    
    return string

In [41]:
def split_readmes(df):
    """
    Takes in a dataframe and performs a 70/15/15 split. Outputs a train, validate, and test dataframe
    """
    # Perfrom a 70/15/15 split
    train_val, test = train_test_split(df, test_size=.15, random_state=123)
    train, validate = train_test_split(train_val, test_size=.17, random_state=123)
    
    return train, validate, test

In [42]:
def prep_readmes(df, col="readme_content"):
    """
    Takes in the dataframe and the column name that contains the corpus data, creates a column of cleaned data, then uses that 
    to create a column without stopwords that is lemmatized, performs a train-validate-test split, performs an x-y split, and
    returns x and y train, x and y validate, and x and y test.
    """
    # Create the cleaned column

    cleaned_row = []
    for i in df.readme_content.values:
        cleaned_row.append(clean(i))
    df = df.assign(cleaned_content=cleaned_row)
#     df['cleaned'] = df[col].apply(lambda x: clean(x))
    df['lemmatized'] = df['cleaned_content'].apply(lambda x: lemmatize(remove_stopwords(x)))
    
    # Split the dataframe (70/15/15)
    train, validate, test = split_readmes(df)
    
#     # perform x-y split
#     x_train, y_train = train.drop(columns=('language')), train.language
#     x_validate, y_validate = validate.drop(columns=('language')), validate.language
#     x_test, y_test = test.drop(columns=('language')), test.language
    
    return train, validate, test

In [44]:
train, val, test = prep_readmes(python_df)

In [45]:
train

,repo_name,url,language,readme_content,cleaned_content,lemmatized
55,josephmisiti/awesome-machine-learning,https://github.com/josephmisiti/awesome-machin...,Python,Awesome Machine Learning A curated list of aw...,awesome machine learning a curated list of aw...,awesome machine learning curated list awesome ...
31,Significant-Gravitas/Auto-GPT,https://github.com/Significant-Gravitas/Auto-G...,Python,Auto-GPT: An Autonomous GPT-4 Experiment💡 Get ...,autogpt an autonomous gpt4 experiment get help...,autogpt autonomous gpt4 experiment get help qa...
14,josephmisiti/awesome-machine-learning,https://github.com/josephmisiti/awesome-machin...,Python,Awesome Machine Learning A curated list of aw...,awesome machine learning a curated list of aw...,awesome machine learning curated list awesome ...
78,PaddlePaddle/PaddleOCR,https://github.com/PaddlePaddle/PaddleOCR/blob...,Python,English | 简体中文 | हिन्दी | 日本語 | 한국인 | Pу́сский...,english p ...,english p paddleocrocr 202387 paddleocr releas...
46,bregman-arie/devops-exercises,https://github.com/bregman-arie/devops-exercis...,Python,DevOps 面试问题ℹ️ 本仓库包含各种 DevOps 相关主题的面试问题📊 当前有 ...,devops i devops 413 devops devops ...,devops devops 413 devops devops pr devops jenk...
...,...,...,...,...,...,...
44,ultralytics/yolov5,https://github.com/ultralytics/yolov5/blob/mas...,Python,English | 简体中文 ...,english ...,english yolov5 world's loved vision ai represe...
40,public-apis/public-apis,https://github.com/public-apis/public-apis/blo...,Python,Public APIs A collective list of free A...,public apis a collective list of free a...,public apis collective list free apis use soft...
88,PaddlePaddle/PaddleOCR,https://github.com/PaddlePaddle/PaddleOCR/blob...,Python,English | 简体中文 | हिन्दी | 日本語 | 한국인 | Pу́сский...,english p ...,english p paddleocrocr 202387 paddleocr releas...
87,fxsjy/jieba,https://github.com/fxsjy/jieba/blob/master/REA...,Python,"jieba“结巴”中文分词：做最好的 Python 中文分词组件\""Jieba\"" (Chi...",jieba python jieba chinese for to stutter chin...,jieba python jieba chinese stutter chinese tex...


In [46]:
train.shape, val.shape, test.shape

((62, 6), (13, 6), (14, 6))